In [1]:
import numpy as np
import pandas
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor,ExtraTreesRegressor,BaggingRegressor
URL = 'C:\Users\Kowshik\Desktop\Lighthouse\Consolidated data\COL.xlsx'
df = pandas.read_excel(URL,sheetname=1)

In [2]:
df_CH = df
import pandas as pd 
#Dropping NAN Values
df_CH = df_CH[df_CH.ProgrammeGenre.notnull()]
df_CH = df_CH[df_CH.ProgrammeTheme.notnull()]
####################
#Dropping AD_Break values
df_CH = df_CH[df_CH.EvType != 'Ad Break'] 
###################################

#Taggin Teleshopping and movies into Movie and Teleshopping
Description = df_CH['Description'].values
for i in range(len(df_CH['Description'].values)):
    if df_CH['ProgrammeTheme'].values[i] == 'Movies / Plays':
         Description[i] = 'Movies'
df_CH['Description'] = Description                              
for i in range(len(df_CH['Description'].values)):
    if df_CH['ProgrammeTheme'].values[i] == 'Teleshopping':
         Description[i] = 'Teleshopping'
df_CH['Description'] = Description

################################################################

#Prior and Subsequent programs 
priorProgram = []
subsequentProgram = []
#df_CH['EvType'].values
for i in range(int(len(df_CH['EvType'].values))):
    if df_CH['EvType'].values[i] == 'Promo':
        for j in range(i,0,-1):
            if df_CH['EvType'].values[j] == 'Program':
                priorProgram.append(df_CH['Description'].values[j])
                break
for i in range(int(len(df_CH['EvType'].values))):
    if df_CH['EvType'].values[i] == 'Promo':
        for j in range(i,int(len(df_CH['EvType'].values))):
            if df_CH['EvType'].values[j] == 'Program':
                subsequentProgram.append(df_CH['Description'].values[j])
                break
df_CH = df_CH[df_CH.EvType != 'Program']
df_CH['priorProgram'] = priorProgram #-------------------------------------------------Priorprograms---------->
df_CH['subsequentProgram'] =  subsequentProgram#--------------------------------------subprograms--------->
df_CH  = df_CH[df_CH.priorProgram.notnull()]
df_CH  = df_CH[df_CH.subsequentProgram.notnull()]

##################################################################
#Categerizing the channels 
URL = 'C:\Users\Kowshik\Desktop\Lighthouse\Consolidated data\COL_P.xlsx'
df_PR = pandas.read_excel(URL)
df_PR = df_PR[['Description','Impressions','ProgrammeTheme']]
Description = df_PR['Description'].values
for i in range(len(df_PR['Description'].values)):
    if df_PR['ProgrammeTheme'].values[i] == 'Movies / Plays':
         Description[i] = 'Movies'
df_PR['Description'] = Description
for i in range(len(df_PR['Description'].values)):
    if df_PR['ProgrammeTheme'].values[i] == 'Teleshopping':
         Description[i] = 'Teleshopping'
df_PR['Description'] = Description
A = np.unique(df_PR['Description'].values)
PI = {}
for i in A:
    PI[i] = []
    for j in range(len(df_PR['Description'].values)):
        if df_PR['Description'].values[j] == i:
            PI[i].append(df_PR['Impressions'].values[j]) 
PI_AVG = {}
for i in A:
    PI_AVG[i] = np.mean(PI[i])
    
clus = []
for i in PI_AVG.values()[1:]:
    clus.append([i])
from sklearn.cluster import KMeans
clu = KMeans(n_clusters=6,n_init = 100,precompute_distances=False,n_jobs=1) #----------------- NO OF CLUSTERS ------------------>
clu.fit(clus)
Description_classified_prior = df_CH['priorProgram'].values
for i in range(len(df_CH['priorProgram'].values)):
    Description_classified_prior[i] = clu.predict(PI_AVG[df_CH['priorProgram'].values[i]])[0]
    
Description_classifiedsubsequent =df_CH['subsequentProgram'].values
for i in range(len(df_CH['subsequentProgram'].values)):
    Description_classifiedsubsequent[i] = clu.predict(PI_AVG[df_CH['subsequentProgram'].values[i]])[0]
df_CH['priorProgram'] = Description_classified_prior
df_CH['subsequentProgram'] = Description_classifiedsubsequent
df_CH.priorProgram= df_CH.priorProgram.astype(str) 
df_CH.subsequentProgram= df_CH.subsequentProgram.astype(str)

##########################
#Deleting and string conversion
df_CH['priorProgram'] = Description_classified_prior #-----------------------------------------------------------classified prior----------->
df_CH['subsequentProgram'] =  Description_classifiedsubsequent#---------------------------------------------------classifiedsub------->
df_CH.Date = df_CH.Date.astype(str)                    #Changing the Date as String 
df_CH.ProgrammeTheme= df_CH.ProgrammeTheme.astype(str) #Changing the Programme Theme in String for encoding 
df_CH.ProgrammeGenre= df_CH.ProgrammeGenre.astype(str) #Changing the Programme Gerne in String for encoding 
df_CH.PromoType= df_CH.PromoType.astype(str)           #Changing the Promo Type in String for encoding 
df_CH.PromoCategory= df_CH.PromoCategory.astype(str)   #Changing the Promo Category in String for encoding
df_CH.StartTime= df_CH.StartTime.astype(str) 
df_CH.EndTime= df_CH.EndTime.astype(str)
df_CH.Length= df_CH.Length.astype(str)
del df_CH['BreakCode']                                  
del df_CH['Description'] 
del df_CH['EvType']
del df_CH['PromoCategory']
del df_CH['EndTime']
del df_CH['Level'] 

Y_CH_1 = np.array(df_CH.Impressions)                       #Creating a array for impression Prediction
Y_CH_2 = np.array(df_CH.Rch)
del df_CH['rat'] 
del df_CH['Shr'] 
del df_CH['Rch_000'] 
del df_CH['Rch'] 
del df_CH['Impressions']
##########################
#Length Dealing
ftr = [3600,60,1]
leng_seconds = [int(sum([a*b for a,b in zip(ftr, map(int,i.split(':')))])) for i in df_CH['Length'].values]
df_CH['Length'] = leng_seconds#----------------------------------------------------------------------------------len in Seconds----------------->

###########################
#Date Dealing
month = [map(int,i.split('-'))[1] for i in df_CH['Date'].values]
day = [map(int,i.split('-'))[2] for i in df_CH['Date'].values]
df_CH['Month'] = month       #----------------------------------------------------------------------------------Month--------->
df_CH['day'] = day           #------------------------------------------------------------------------------------Day--------->
#####################################
#Weekday dealing
weekday = []
import datetime
for i in df_CH['Date'].values:
    year, month, day = (int(x) for x in i.split('-'))    
    ans = datetime.date(year, month, day)
    weekday.append(ans.strftime("%A"))
df_CH['weekday'] = weekday  #---------------------------------------------------------------------------------------WeekdaysX------------>
#################################################################
#Binary weekdays dealing
binary_weekday = weekday[:]
for i in range(len(weekday)):
    if weekday[i] == 'Saturday':
        binary_weekday[i] = 'Yes'
    elif weekday[i] == 'Sunday':
        binary_weekday[i] = 'Yes'
    else:
        binary_weekday[i] = 'No'
df_CH['B_weekday'] = binary_weekday #--------------------------------------------------------------------------------Binary weekend------>
##################################################################
del df_CH['Date'] #-----------------------------------------------------------------------------------------------Deleting Date------
#StartTime Dealing
Starttime = df_CH['StartTime'].values
for i in range(len(df_CH['StartTime'].values)):
    if len(df_CH['StartTime'].values[i].split()) == 2:     
        Starttime[i] =  df_CH['StartTime'].values[i].split()[1]
df_CH['StartTime'] = Starttime #------------------------------------------------------------------------------------------Start time
df_CH['hour'] = [map(int,i.split(':'))[0] for i in df_CH['StartTime'].values]#--------------------hour------------------------------->
df_CH.hour= df_CH.hour.astype(str)
df_CH['StartTime'] = [map(int,i.split(':'))[0]+ map(int,i.split(':'))[1]/60.0+map(int,i.split(':'))[1]/3600.0 for i in df_CH['StartTime'].values]



C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\numpy\lib\arraysetops.py:198: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
C:\Users\Kowshik\AppData\Local\Enthought\Canopy\User\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [3]:
S = PI_AVG.values()
S.sort()

In [4]:
for i in S[1:]:
    print clu.predict(i)

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[2]
[2]
[2]
[2]
[2]
[5]
[5]
[5]


In [20]:
#Decision channel
df_CH_1 = df_CH[:]
#del df_CH_1['StartTime'] #-------------------------------------------------------------------------------------------deleting starttime------>#
del df_CH_1['priorProgram'] #---------------------------------------------------------------------------------- -----prior programme--------->
del df_CH_1['subsequentProgram']#-------------------------------------------------------------------------- ----- ---subsequent programme
#del df_CH_1['Length'] #---------------------------------------------------------------------------------------------del length -------------->
#del df_CH_1['ProgrammeTheme'] #-------------------------------  ---------------------------------------------------- programe theme------->
#del df_CH_1['ProgrammeGenre'] #------------------------------------------------------------------------------------- ProgrammeGenre------->
del df_CH_1['PromoType']      #-------------------------------------------------------------------------------------promotype------->
del df_CH_1['B_weekday'] #---------------------------------------------------------------  --------------------------B-weekend------>
del df_CH_1['hour'] #-----------------------------------------------------------------------  ------------------------Hour--------->
#del df_CH_1['weekday'] #---------------------------------------------------------------------- -----------------------weekday-------->
#del df_CH_1['day'] #-------------------------------------------------------------------------- ------------------------day------->
del df_CH_1['Month'] #-------------------------------------------------------------------------------------------------Month------>

In [13]:
x_t = []
for j in np.unique(df_CH_1['priorProgram'].values):
        for k in np.unique(df_CH_1['subsequentProgram'].values):
            for l in np.unique(df_CH_1['B_weekday'].values):
                for m in np.arange(2,25,0.5):
                    for q in np.unique(df_CH_1['day'].values):
                        D = {}
                        D['StartTime'] = m
                        D['B_weekday'] = l
                        D['subsequentProgram'] = k
                        D['priorProgram'] = j
                        D['day'] = q
                        x_t.append(D)                                 

In [ ]:
import random
x_testing = []
for i in range(5000):#len(df_CH_1['ProgrammeGenre'].values)):
    d = {}
    d['B_weekday'] = random.choice(np.unique(df_CH_1['B_weekday'].values))
    d['StartTime'] = random.choice(np.arange(2,25,0.1))
    d['priorProgram'] = random.choice(np.unique(df_CH_1['priorProgram'].values))
    d['subsequentProgram'] = random.choice(np.unique(df_CH_1['subsequentProgram'].values))
    #d['Length'] = random.choice(np.unique(df_CH_1['Length'].values))
    #d['ProgrammeTheme'] = random.choice(np.unique(df_CH_1['ProgrammeTheme'].values))
    #d['ProgrammeGenre'] = random.choice(np.unique(df_CH_1['ProgrammeGenre'].values))
    #d['hour'] = random.choice(np.unique(df_CH_1['hour'].values))
    #d['day'] = random.choice(np.unique(df_CH_1['day'].values))
    #d['Month'] = random.choice(np.unique(df_CH_1['Month'].values))
    #d['PromoType'] = random.choice(np.unique(df_CH_1['PromoType'].values))
    #d['weekday'] = random.choice(np.unique(df_CH_1['weekday'].values))
    x_testing.append(d)   

In [14]:
#ENCODING
X_dict = df_CH_1.to_dict('records')
X_en = []
X_en.extend(X_dict)
#One Hot Encoding 
enc = DictVectorizer()
X_encoded = enc.fit_transform(X_en)
X_tes = enc.fit_transform(x_t)

In [21]:
#ENCODING
X_dict = df_CH_1.to_dict('records')
X_en = []
X_en.extend(X_dict)
#One Hot Encoding 
enc = DictVectorizer()
X_encoded = enc.fit_transform(X_en)
#Validating set and training set
train_split = int(0.15*df_CH_1.shape[0])
X_vald =  X_encoded[0:train_split]
Y_vald_1 = Y_CH_1[0:train_split]
Y_vald_2 = Y_CH_2[0:train_split]
X_train = X_encoded[train_split:]
Y_train_1 = Y_CH_1[train_split:]
Y_train_2 = Y_CH_2[train_split:]

In [5]:
#ENCODING_2
X_dict = df_CH_1.to_dict('records')
X_en = []
X_en.extend(X_dict)
#One Hot Encoding 
enc = DictVectorizer()
X_encoded = enc.fit_transform(X_en)
#Validating set and training set
train_split = int(0.85*df_CH_1.shape[0])
X_vald =  X_encoded[train_split:]
Y_vald_1 = Y_CH_1[train_split:]
Y_vald_2 = Y_CH_2[train_split:]
X_train = X_encoded[0:train_split:]
Y_train_1 = Y_CH_1[0:train_split:]
Y_train_2 = Y_CH_2[0:train_split:]

In [22]:
#Gradient Boosting
reg_grad = GradientBoostingRegressor(learning_rate = 0.01, n_estimators  = 1000, loss = 'huber')
reg_grad.fit(X_encoded,Y_CH_1)
Y_grad_1 = reg_grad.predict(X_tes.toarray())

ValueError: Number of features of the model must  match the input. Model n_features is 38 and  input n_features is 16 

In [16]:
Y_grad_1 = list(Y_grad_1) 
Y  =Y_grad_1[:]

In [17]:
Y.sort()

In [18]:
Y_grad_1.index(Y[len(Y)-1])

42544

In [7]:
#Gradient Boosting
reg_grad = GradientBoostingRegressor(learning_rate = 0.01, n_estimators  = 1000, loss = 'huber')
reg_grad.fit(X_train,Y_train_1)
Y_grad_1 = reg_grad.predict(X_vald.toarray())
reg_grad.score(X_vald.toarray(),Y_vald_1)

0.69630481418859169

In [ ]:
#Extra Trees
reg_ET_2 = ExtraTreesRegressor()
reg_ET_2.fit(X_train,Y_train_2)
Y_ET_2 = reg_ET_2.predict(X_vald)
reg_ET_2.score(X_vald,Y_vald_2)

In [ ]:
#Gradient Boosting
reg_grad = GradientBoostingRegressor()
reg_grad.fit(X_train,Y_train_1)
Y_grad_2 = reg_grad.predict(X_vald.toarray())
reg_grad.score(X_vald.toarray(),Y_vald_1)

In [ ]:
#Random
reg_RFR = RandomForestRegressor(n_jobs= -1)
reg_RFR.fit(X_train,Y_train_1)
Y_RFR_1 = reg_grad.predict(X_vald.toarray())
reg_RFR.score(X_vald.toarray(),Y_vald_1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#OUR DATA
A = 600
X = np.arange(A)
Y = Y_vald_1[:A]
x = np.arange(A)
y = Y_grad_1[:A]
####################
#Visuvalization
canvas = plt.figure()
rect = canvas.patch
rect.set_facecolor('white')
sp1 = canvas.add_subplot(1,1,1, axisbg='w')
sp1.plot(x, y, 'Red', linewidth=2)
sp1.plot(X, Y, 'Black', linewidth=1)

sp1.patch.set_facecolor('white') #------> background colour

sp1.tick_params(axis='x', colors='green') #---------> X-value colour
sp1.tick_params(axis='y', colors='red') #----------> y-value colour

sp1.spines['bottom'].set_color('black')        #Colour of spines 
sp1.spines['top'].set_visible('False')
sp1.spines['left'].set_color('black')
sp1.spines['right'].set_visible('False')

sp1.set_title('Impressions vs time Using Extra Classifier', color='red')
sp1.set_xlabel('Time,BLUE -Actual', color='red')      #Colour of title and lables 
sp1.set_ylabel('Impression', color='red')
plt.plot(X,Y)
plt.plot(x,y)

plt.tight_layout()
plt.show()

In [15]:
import numpy as np
import matplotlib.pyplot as plt
#OUR DATA

X = np.arange(len(grp))
Y = grp

####################
#Visuvalization
canvas = plt.figure()
rect = canvas.patch
rect.set_facecolor('white')
sp1 = canvas.add_subplot(1,1,1, axisbg='w')
sp1.plot(X, Y, 'Black', linewidth=1)

sp1.patch.set_facecolor('white') #------> background colour

sp1.tick_params(axis='x', colors='green') #---------> X-value colour
sp1.tick_params(axis='y', colors='red') #----------> y-value colour

sp1.spines['bottom'].set_color('black')        #Colour of spines 
sp1.spines['top'].set_visible('False')
sp1.spines['left'].set_color('black')
sp1.spines['right'].set_visible('False')

sp1.set_title('Impressions vs time Using Extra Classifier', color='red')
sp1.set_xlabel('Time,BLUE -Actual', color='red')      #Colour of title and lables 
sp1.set_ylabel('Impression', color='red')
plt.plot(X,Y)

plt.tight_layout()
plt.show()

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

def mov(x):
    return Y_CH_1[x]
x = np.arange(0, 400, 1)
line, = ax.plot(x, mov(x))
ax.set_xlabel('Time,BLUE -Actual', color='red')
ax.patch.set_facecolor('black') 

def animate(i):
    line.set_ydata(mov(x + 4*i))  # Speed deciding
    return line,

def init():
    line.set_ydata(np.ma.array(x, mask=True))
    return line,
fig.patch.set_facecolor('white')
ani = animation.FuncAnimation(fig, animate, np.arange(1, len(Y_CH_1)), init_func=init,
                              interval=25, blit=True)
plt.show()

In [ ]:
grp = []
for i in range(len(Y)):
    print x_t[Y_grad_1.index(Y[len(Y)-1-i])]
    grp.append(Y[len(Y)-1-i])

{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 13}
{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 13}
{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 2}
{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 2}
{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 2}
{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 2}
{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 2}
{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 2}
{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 2}
{'subsequentProgram': '2', 'B_weekday': 'Yes', 'priorProgram': '2', 'StartTime': 21.0, 'day': 2}
{'subsequentProgram': '2', '

In [74]:
from sklearn.metrics import r2_score
r2_score(Y_vald_1,Y_grad_1)

0.89598824090462981

In [77]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
mean_absolute_error(Y_grad_1,Y_vald_1)

380.37784755727199

In [78]:
mean_squared_error(Y_grad_1,Y_vald_1)

684831.55321205733